In [1]:
import dash
from dash import dcc, html, Input, Output
import pandas as pd
import plotly.express as px
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv("steam-200k.csv") 

user_game_matrix = data.pivot_table(index='user_id', columns='game_name', values='hour', fill_value=0)

user_similarity = pd.DataFrame(cosine_similarity(user_game_matrix), 
                                index=user_game_matrix.index, 
                                columns=user_game_matrix.index)

game_similarity = pd.DataFrame(cosine_similarity(user_game_matrix.T), 
                                index=user_game_matrix.columns, 
                                columns=user_game_matrix.columns)

total_users = data['user_id'].nunique()
total_games = data['game_name'].nunique()
total_interactions = len(data)
total_hours = data['hour'].sum()

app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Game Recommendation Dashboard", style={'text-align': 'center'}),

    html.Div([
        html.H3("Dataset Summary"),
        html.P(f"Total Unique Users: {total_users}"),
        html.P(f"Total Unique Games: {total_games}"),
        html.P(f"Total Interactions: {total_interactions}"),
    ], style={'text-align': 'center', 'margin-bottom': '20px'}),

    html.Div([
        html.Label("Select User ID:"),
        dcc.Dropdown(
            id='user-dropdown',
            options=[{'label': user, 'value': user} for user in user_game_matrix.index],
            value=user_game_matrix.index[0]
        ),
        html.Div(id='recommendation-output', style={'margin-top': '20px'}),
        dcc.Graph(id='recommendation-heatmap', style={'margin-top': '20px'}),
    ], style={'width': '50%', 'margin': '0 auto'}),

    html.Div([
        html.Label("Select Game:"),
        dcc.Dropdown(
            id='game-dropdown',
            options=[{'label': game, 'value': game} for game in user_game_matrix.columns],
            value=user_game_matrix.columns[0]
        ),
        dcc.Graph(id='game-similarity-graph'),
    ], style={'width': '50%', 'margin': '20px auto'}),

    html.Div([
        html.H3("Top 10 Games by Total Hours Played (All Data)", style={'text-align': 'center'}),
        dcc.Graph(id='top-games-graph'),

        html.H3("Top Games by Total Hours Played (Selected User)", style={'text-align': 'center', 'margin-top': '20px'}),
        dcc.Graph(id='user-top-games-graph'),
    ], style={'margin-top': '20px'}),
])

@app.callback(
    [Output('recommendation-output', 'children'),
     Output('recommendation-heatmap', 'figure')],
    [Input('user-dropdown', 'value')]
)
def recommend_for_user_with_heatmap(user_id):
    
    similar_users = user_similarity[user_id].sort_values(ascending=False).head(10)
    
    similar_users_data = user_game_matrix.loc[similar_users.index]
    game_scores = similar_users_data.mean(axis=0)
    user_games = user_game_matrix.loc[user_id]
    recommendations = game_scores[user_games == 0].sort_values(ascending=False).head(10)

    recommendation_text = html.Div([
        html.H3(f"Recommendations for User {user_id}:"),
        html.Ul([html.Li(game) for game in recommendations.index])
    ])

    recommended_game_names = recommendations.index
    heatmap_data = game_similarity.loc[recommended_game_names, recommended_game_names]

    heatmap_fig = px.imshow(
        heatmap_data.values,
        labels=dict(x="Games", y="Games", color="Similarity"),
        x=heatmap_data.columns,
        y=heatmap_data.index,
        color_continuous_scale="Viridis"
    )

    heatmap_fig.update_layout(
        title=f"Correlation Heatmap for Recommended Games (User {user_id})",
        xaxis=dict(
            tickangle=45,
            tickfont=dict(size=10) 
        ),
        yaxis=dict(
            tickfont=dict(size=10) 
        ),
        margin=dict(l=100, r=100, t=100, b=150) 
    )

    return recommendation_text, heatmap_fig

@app.callback(
    Output('game-similarity-graph', 'figure'),
    [Input('game-dropdown', 'value')]
)
def plot_similar_games(game_name):
    similar_games = game_similarity[game_name].sort_values(ascending=False).head(10)
    fig = px.bar(similar_games, x=similar_games.index, y=similar_games.values,
                 labels={'x': 'Game Name', 'y': 'Similarity'},
                 title=f"Top 10 Similar Games to {game_name}")
    return fig

@app.callback(
    Output('top-games-graph', 'figure'),
    [Input('game-dropdown', 'value')]
)
def plot_top_games(_):
    top_games = user_game_matrix.sum(axis=0).sort_values(ascending=False).head(10)
    fig = px.bar(top_games, x=top_games.index, y=top_games.values,
                 labels={'x': 'Game Name', 'y': 'Total Hours Played'},
                 title="Top 10 Games by Total Hours Played")
    return fig

@app.callback(
    Output('user-top-games-graph', 'figure'),
    [Input('user-dropdown', 'value')]
)
def plot_user_top_games(user_id):

    user_games = user_game_matrix.loc[user_id]

    user_top_games = user_games[user_games > 0].sort_values(ascending=False).head(10)

    user_top_games = user_top_games.astype(float)
    
    if user_top_games.empty:
        fig = px.bar(title=f"User {user_id} Has No Game Data")
    else:
        fig = px.bar(
            user_top_games,
            x=user_top_games.index,
            y=user_top_games.values,
            labels={'x': 'Game Name', 'y': 'Total Hours Played'},
            title=f"Top Games by Total Hours Played for User {user_id}"
        )

    fig.update_layout(
        xaxis=dict(tickangle=45),
        yaxis=dict(tickformat=".0f"),
        height=400
    )

    return fig

if __name__ == '__main__':
    app.run_server(debug=True, port=4001)